## Need to run:
coevolve python package (currently only installed locally in /data2/isshamie/coevolve/coevolve. Go to that folder and run pip install -e .)

1. parameters folder. The evc template is in there and the parameter file that points to the genome information (in case that changes).  
2. data/raw is the folder that contains that genome files  
3. data/notebook with this file and Covid19 - Alignment and Couplings Output.ipynb

### Parameter file (should be in parameters folder)

In [1]:
main_dir = "../"
param = 'sars_cov2.yaml'

### 0.1 Import 

In [2]:
import pandas as pd
import os
import glob
from Bio import SeqIO, Seq, pairwise2, SeqRecord, Align
from os.path import join, basename
import pickle
import numpy as np
from tqdm import tqdm
from numpanpar import parallel_ar as parar
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

/home/isshamie/software/anaconda2/envs/coevolve/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from config import FIGURES_DIR, ROOT_DIR, PARAMS_DIR, RESULTS, CONTACT_FIGURES_DIR, NUM_CORES
from coevolve.msa import clustalw_query
from coevolve.config import CLUSTALW_DIR, CLUSTALW_CMD, CLUSTALW_CLEAN_DIR
from evcouplings.utils import read_config_file, write_config_file
from coevolve.couplings.running_jobs_complexes import run_evc
from coevolve.couplings.couplings_vis import construct_monomer_contact_matrix
from coevolve.data.data_io import read_msa

Project Directory: /data2/isshamie/covid/covid_03302020

Config paths:
__name__ config
__package__ 
__file__ /data2/isshamie/covid/covid_03302020/notebook/config.py
__cached__ /data2/isshamie/covid/covid_03302020/notebook/__pycache__/config.cpython-37.pyc
path /data2/isshamie/covid/covid_03302020/notebook/config.py
DATA_DIR /data2/isshamie/covid/covid_03302020/data
RESULTS /data2/isshamie/covid/covid_03302020/data/processed
FIGURES_DIR /data2/isshamie/covid/covid_03302020/figures
CONTACT_FIGURES_DIR /data2/isshamie/covid/covid_03302020/figures/contact
PAIR_COUPLINGS_FIGURES_DIR /data2/isshamie/covid/covid_03302020/figures/pair_couplings
GLYCO_PAIR_COUPLINGS_FIGURES_DIR /data2/isshamie/covid/covid_03302020/figures/glycosites_pair_couplings
PARAMS_DIR /data2/isshamie/covid/covid_03302020/parameters
d /data2/isshamie/covid/covid_03302020/figures/glycosites_pair_couplings
UNIPROT_MAP /data2/isshamie/covid/covid_03302020/data/external/biomart/mart_export_uniprot.txt
GENE_MAP /data2/isshamie

### Setup folders and files

In [4]:
RAW = join(main_dir,"data","raw")
outdir = join(main_dir,"data","processed", "align_fromErick")
param_dir = join(main_dir,"parameters")
evzoom_template_dir = join(ROOT_DIR, "software", "EVzoom")


seqs_dir = join(outdir,"seqs")
align_dir = join(seqs_dir,"S_protein_align")
align_concat_dir = join(seqs_dir,"S_protein_align_full")

msa_out = join(outdir, "msa")
msa_f = join(msa_out,"S_protein.msa.fasta")
anno_f = join(seqs_dir, "covid_annotations.csv")
clean = join(msa_out,"clean")

ev_dir = join(outdir,"evc")
contact_dir = join(outdir, "contact")
ev_params_dir = join(outdir,"ev_params")

evzoom_dir = join(outdir,"evzoom")
evc_template_f = join(param_dir,'evc',"template.yaml")

#clustalw_query.create_clean_clustalw_names(clean, [msa_f])

msa_clean_f = join(clean, os.path.basename(msa_f))

NUM_CORES = 32
if not os.path.exists(outdir):
    os.mkdir(outdir)

if not os.path.exists(seqs_dir):
    os.mkdir(seqs_dir)
    
if not os.path.exists(align_dir):
    os.mkdir(align_dir)
    
if not os.path.exists(align_concat_dir):
    os.mkdir(align_concat_dir)
    
if not os.path.exists(msa_out):
    os.mkdir(msa_out)
    
if not os.path.exists(msa_out):
    os.mkdir(msa_out)

if not os.path.exists(clean):
    os.mkdir(clean)

    
if not os.path.exists(ev_dir):
    os.mkdir(ev_dir)
    
if not os.path.exists(ev_params_dir):
    os.mkdir(ev_params_dir)
    
if not os.path.exists(param_dir):
    os.mkdir(param_dir)
    
if not os.path.exists(evzoom_dir):
    cmd = f'cp -r {evzoom_template_dir} {evzoom_dir}'
    print(cmd)
    os.system(cmd)

if not os.path.exists(contact_dir):
    os.mkdir(contact_dir)
    

cp -r /data2/isshamie/covid/covid_03302020/software/EVzoom ../data/processed/align_fromErick/evzoom


## 1. Load:
- A) Genomic fasta sequences, 
- B) Reference Genomic Annotation (GCF_009858895.2)
- C) Reference Gene Annotation (NC_045512.2)
- D) S-protein fasta protein sequence

In [5]:
config = read_config_file(join(param_dir, param))
config

{'name': 'sars_cov2',
 'ref_fasta': 'NC_045512.2.fasta',
 'genomes_fasta': '2020-03-30-gisaid-seqs.fasta',
 'S_protein_name': 'YP_009724390.1',
 'annotation': 'GCF_009858895.2_ASM985889v3_genomic.gff'}

In [6]:
# ref_fasta = SeqIO.read(os.path.join(RAW,'NC_045512.2.fasta'), format="fasta")
# anno = pd.read_csv(glob.glob(RAW+"/*.gff")[0],sep="\t", comment="#", header=None)
# all_genomic_sequences = list(SeqIO.parse(os.path.join(RAW,"gisaid_cov2020_sequences.fasta"), "fasta"))

# S_protein_name = "YP_009724390.1"
# S_protein_fasta = SeqIO.read(os.path.join(RAW, S_protein_name+".fasta"),format='fasta')
# S_protein_fasta = str(S_protein_fasta.seq)


ref_fasta = SeqIO.read(os.path.join(RAW,config['ref_fasta']), format="fasta")
anno = pd.read_csv(join(RAW,config['annotation']),sep="\t", comment="#", header=None)
all_genomic_sequences = list(SeqIO.parse(os.path.join(RAW,config["genomes_fasta"]), "fasta"))

S_protein_name = config['S_protein_name']
S_protein_fasta = SeqIO.read(os.path.join(RAW, S_protein_name+".fasta"),format='fasta')
S_protein_fasta = str(S_protein_fasta.seq)


## 2. Preprocess

### a. Filter annotation for the S-protein.
Confirm the protein sequence aligns with the genome annotation.
- Extract the genomic region from annotation and take the sequence
- Translate sequence
- See if aligns with the Protein fasta sequence



See below that there are no introns,  so we can just use that sequence

In [7]:
anno_S = anno[anno[8].str.contains(S_protein_name)]

#Extract protein sequence from this region
S_gene_seq = ref_fasta.seq[anno_S[3].values[0]-1: anno_S[4].values[0]]
annotation_protein_sequence = str(Seq.translate(S_gene_seq))
annotation_protein_sequence = annotation_protein_sequence.replace("*","")

S_gene_seq = SeqRecord.SeqRecord(S_gene_seq, id=S_protein_name)
SeqIO.write(S_gene_seq, open(join(seqs_dir,"S_gene.fasta"),'w'),format='fasta')

1

In [8]:
annotation_protein_mismatch = []
for ind, val in enumerate(annotation_protein_sequence):
    if S_protein_fasta[ind] != val:
        annotation_protein_mismatch.append((ind, val))
annotation_protein_mismatch

[]

### b. remove any sequence less than 29kb

In [9]:
print(f"Number of sequences before filtering: {len(all_genomic_sequences)}")
all_genomic_sequences_filt = []
for seq in all_genomic_sequences:
    if len(seq.seq) >= 29000:
        all_genomic_sequences_filt.append(seq)
print(f"Number of sequences after filtering for at least 29kb: {len(all_genomic_sequences_filt)}")

Number of sequences before filtering: 2909
Number of sequences after filtering for at least 29kb: 2794


### c. Remove duplicates sequences

In [10]:
inds_to_remove = []
curr_seqs = []

for ind,seq in enumerate(all_genomic_sequences_filt):
    if str(seq.seq) in curr_seqs:
        inds_to_remove.append(seq.id)
    else:
        curr_seqs.append(str(seq.seq))

all_genomic_sequences_filt = list(filter(lambda x: x.id not in inds_to_remove, all_genomic_sequences_filt))
print(f"Number of sequences after removing duplicate sequences: {len(all_genomic_sequences_filt)}")


Number of sequences after removing duplicate sequences: 2463


### d. Remove duplicate IDs

In [11]:
inds_to_remove = []
curr_names = []

for ind,seq in enumerate(all_genomic_sequences_filt):
    if seq.id in curr_names:
        inds_to_remove.append(seq.id)
    else:
        curr_names.append(seq.id)

all_genomic_sequences_filt = list(filter(lambda x: x.id not in inds_to_remove, all_genomic_sequences_filt))
print(f"Number of sequences after removing duplicate sequences: {len(all_genomic_sequences_filt)}")


Number of sequences after removing duplicate sequences: 2460


### e. Remove animals 

In [12]:
all_genomic_sequences_filt = list(filter(lambda x: not ("pangolin" in x.id or "bat" in x.id), all_genomic_sequences_filt))
print(len(all_genomic_sequences_filt))

2451


## 3. Load the genome annotations from Erick, and for each genome in all_genomic_sequences_filt, get the Spike protein, translate it, and save it 

In [13]:
def remove_chars(orig_str, chars = ["|",")","(","*"]):
    return orig_str.replace("|","_").replace(")","").replace("*","_").replace("(","").replace(" ","_").replace("-","_").replace("/","").replace(">","")

In [14]:
all_annotations=pd.read_csv(join(RAW, 'erick', 'GISAID_Annotations.csv'))
all_annotations.head()

all_annotations = all_annotations[all_annotations["Id"] == "gene-S"]
all_annotations = all_annotations.set_index("Location")
print("Any duplicated IDs:", all_annotations.index.duplicated().any())
all_annotations.head()

Any duplicated IDs: False


,Id,Type,Start,End,Strand
Location,,,,,
hCoV-19/USA/WA-S88/2020|EPI_ISL_417141|2020-03-01,gene-S,gene,21563,25384,+
hCoV-19/USA/WA-S89/2020|EPI_ISL_417142|2020-02-29,gene-S,gene,21563,25384,+
hCoV-19/USA/WA-S87/2020|EPI_ISL_417140|2020-03-01,gene-S,gene,21563,25384,+
hCoV-19/USA/WA-S92/2020|EPI_ISL_417145|2020-02-29,gene-S,gene,21563,25384,+
hCoV-19/USA/WA-S93/2020|EPI_ISL_417146|2020-02-29,gene-S,gene,21563,25384,+


### No - strand

In [15]:
(all_annotations["Strand"]=="-").any()

False

In [16]:
S_proteins = []
not_seen = []
seq_lengths = []
names = []
for seq in all_genomic_sequences_filt:
    S_gene_location = seq.name
    if seq.name not in all_annotations.index:
        not_seen.append(seq.name)
    elif seq.name in names:
        print("Duplicate!", seq.name)
        continue
    else:
        names.append(seq.name)
        curr = all_annotations.loc[seq.name]
        curr_S_sequence = seq.seq[curr["Start"]-1:curr["End"]]
        #print(curr_S_sequence)
        name = seq.id #remove_chars(seq.id)
        S_proteins.append(SeqRecord.SeqRecord((Seq.translate(curr_S_sequence, stop_symbol="")), id=name, name=name))
        seq_lengths.append(len(str(Seq.translate(curr_S_sequence, stop_symbol=""))))
print(f"Not seen in sequence data: {len(not_seen)}")
print(f"Number of sequences in sequence data: {len(all_genomic_sequences_filt)}")
print(f"Number of unique names: {len(names)}")
#     seq.seq= seq.seq.upper()
#     seq.seq = str(seq.seq).replace("-","")
    
#     curr_out = pairwise2.align.localms(str(seq.seq),str(S_gene_seq.seq), 2, -10, -0.5, -0.1)#, one_alignment=True)
#     curr_align = str(seq.seq)[curr_out[0][3]:curr_out[0][4]]

#     S_all_proteins = [(SeqRecord.SeqRecord(Seq.Seq(Seq.translate(curr_align, stop_symbol="")), id=name))]


Not seen in sequence data: 495
Number of sequences in sequence data: 2451
Number of unique names: 1956


In [17]:
print((np.array(seq_lengths) == 0).sum())
print((np.array(seq_lengths)==1273).all())
sns.countplot(seq_lengths,log=True)

0
False


###  Additional filter to remove any lengths less than 1270

In [18]:
S_proteins = list(filter(lambda x: len(x.seq)>=1270, S_proteins))

In [19]:
all_annotations[all_annotations.index.str.contains("Czech")]

,Id,Type,Start,End,Strand
Location,,,,,
hCoV-19/Czech,gene-S,gene,21556,25377,+


### Save the fasta sequences

In [20]:
S_protein_sequences_f = join(align_concat_dir,"S_protein.fasta")
SeqIO.write(S_proteins, open(S_protein_sequences_f,'w'),format='fasta')


1954

## 5. Plot similarity

## 6. Run MSA on protein using clustalw

In [21]:
clustalw_query.extract_msa(S_protein_sequences_f,msa_out, CLUSTALW_CMD, "isshamie@ucsd.edu",
                to_overwrite=True, web=True,
                local_backup=True, num_cores=NUM_CORES, verbose=True)

S_protein
/home/isshamie/software/anaconda2/envs/coevolve/bin/python /data2/isshamie/coevolve/software/clustalo.py --email isshamie@ucsd.edu --stype protein --outfmt fa --outfile ../data/processed/align_fromErick/msa/S_protein.msa.fasta --sequence  ../data/processed/align_fromErick/seqs/S_protein_align_full/S_protein.fasta
mv ../data/processed/align_fromErick/msa/S_protein.msa.fasta.aln-fasta.fasta ../data/processed/align_fromErick/msa/S_protein.msa.fasta


In [22]:
## Remove characters after space
cmd = f"sed 's/\s.*$//' {msa_f} > {msa_clean_f}"
print(cmd)
os.system(cmd)

sed 's/\s.*$//' ../data/processed/align_fromErick/msa/S_protein.msa.fasta > ../data/processed/align_fromErick/msa/clean/S_protein.msa.fasta


0

## 7. Run EVCouplings
- Create annotation and yaml
- Run 

## See if a target sequence matches the full sequence and use that as target sequence. Otherwise choose randomly

In [23]:
record_dict = SeqIO.to_dict(SeqIO.parse(msa_f, "fasta"))
anno_df = pd.DataFrame(index=record_dict.keys(),columns=["id", "name", "GN", "OS", "Tax", "RepID",
                       "id_raw"],dtype=str)
for ind, val in anno_df.iterrows():
    val[:] = ind
anno_df.to_csv(anno_f, index=None)


In [24]:
def create_annotation_from_fasta(msa_f, anno_f, include_region=False):
    """Create a csv file where each sequence is considered its own organism. 
    Columns needed are ["id", "name", "GN", "OS", "Tax", "RepID",
                       "id_raw"] """
    record_dict = SeqIO.to_dict(SeqIO.parse(msa_f, "fasta"))
    anno_df = pd.DataFrame(index=record_dict.keys(),columns=["id", "name", "GN", "OS", "Tax", "RepID",
                           "id_raw"],dtype=str)
    for ind, val in anno_df.iterrows():
        val[:] = ind
    anno_df.to_csv(anno_f, index=None)
    return

# def run_yaml():
#     configs = [os.path.join(outdir,  "covid.yaml")]
#     print(configs)
#     run_evc(configs, to_overwrite=True)
#     return


## Run with both high coverage of a residue and low 
### Need a fake target sequence, so take the first one, but make sure the minimum_sequence_coverage is 0

In [25]:
protein_seqs = list(SeqIO.parse(join(align_concat_dir,"S_protein.fasta"), "fasta"))

ref_protein = str(Seq.translate(S_gene_seq.seq, stop_symbol=""))
target_seq=""
for seq in protein_seqs:
    if str(seq.seq) == ref_protein:
        target_seq = seq.id
        break
print(target_seq)

hCoV-19/USA/WA-S88/2020|EPI_ISL_417141|2020-03-01


## length of each protein 

In [26]:
import seaborn as sns

s_lengths = []
for seq in protein_seqs:
    s_lengths.append(len(str(seq.seq)))
    
sns.distplot(s_lengths)

/home/isshamie/software/anaconda2/envs/coevolve/lib/python3.7/site-packages/seaborn/distributions.py:340: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, auto=None)


In [27]:
## 
column_thresholds = [75]
redundant_thresholds = [0.99]
#similarity_threshold = 50

files_to_run = []
for col_thresh in column_thresholds:
    for red_thresh in redundant_thresholds:
        name = f"col{col_thresh}_redund{red_thresh}"
        config = read_config_file(evc_template_f, preserve_order=True)
        config["align"]["input_alignment"] = msa_clean_f
        config["align"]["override_annotation_file"] = anno_f
        if target_seq == "":
            seqid = pd.read_csv(anno_f).iloc[100,0] #Just take any name
        else:
            seqid = target_seq
        print(seqid)
        config["align"]["sequence_id"] = seqid
        config["global"]["sequence_id"] = seqid
        config["global"]["sequence_file"] = msa_clean_f
        config["global"]["prefix"] = os.path.join(ev_dir, name)
        config["align"]["minimum_sequence_coverage"] = 0
        config["align"]["minimum_column_coverage"] = col_thresh
        config["align"]["theta"] = red_thresh
        config["global"]["theta"] = red_thresh
        out_f = join(ev_params_dir,name+".yaml")
        files_to_run.append(out_f)
        print(f"Creating file {out_f}")
        write_config_file(out_f, config)
        
run_evc(files_to_run,to_overwrite=True)

hCoV-19/USA/WA-S88/2020|EPI_ISL_417141|2020-03-01
Creating file ../data/processed/align_fromErick/ev_params/col75_redund0.99.yaml
../data/processed/align_fromErick/ev_params/col75_redund0.99.yaml
../data/processed/align_fromErick/evc/col75_redund0.99
{}
HI


/data2/isshamie/coevolve/software/EVcouplings/evcouplings/couplings/pairs.py:593: RuntimeWarning: divide by zero encountered in log
  params[0], params[1]
/data2/isshamie/coevolve/software/EVcouplings/evcouplings/couplings/pairs.py:593: RuntimeWarning: invalid value encountered in multiply
  params[0], params[1]
/data2/isshamie/coevolve/software/EVcouplings/evcouplings/couplings/pairs.py:560: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(cls.mixture_pdf(x, *params)))
/data2/isshamie/coevolve/software/EVcouplings/evcouplings/couplings/pairs.py:532: RuntimeWarning: invalid value encountered in true_divide
  posterior[x > 0] = (1 - p) * f2[x > 0] / P[x > 0]
/data2/isshamie/coevolve/software/EVcouplings/evcouplings/couplings/pairs.py:560: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(cls.mixture_pdf(x, *params)))
/data2/isshamie/coevolve/software/EVcouplings/evcouplings/couplings/pairs.py:593: RuntimeWarning: divide by zero encountered 

## Make EVZoom htmls and move jsons

In [28]:
column_thresholds = [75]
redundant_thresholds = [0.99]

files_to_run = []
for col_thresh in column_thresholds:
    for red_thresh in redundant_thresholds:
        gene_of_interest = f"col{col_thresh}_redund{red_thresh}"
        print(gene_of_interest)
        #Copy over json
        coupling_prefix = os.path.join(ev_dir, gene_of_interest,"couplings",gene_of_interest)
        json_f = coupling_prefix + '_evzoom.json'
        out_json_f = evzoom_dir+"/data/" + os.path.basename(json_f)
        cmd = f'cp {json_f} {out_json_f}'
        print(cmd)
        os.system(cmd)
        
        # Create the html file
        tmp_html = join(evzoom_dir, 'example','evzoom.html')
        out_html = join(evzoom_dir, 'example',gene_of_interest+'.html')
        print(out_html)
        with open(tmp_html, "r") as f:
            page = f.read()
        new_page=""
        line = f'<div id="evzoom-viewer" data-couplings=../data/{basename(json_f)}> </div> '
        for i in page.split("\n"):
            if 'data-couplings' in i:
                new_page = new_page + line + "\n"
            else:
                new_page = new_page + i + "\n"
        new_page = new_page.strip("\n")
        with open(out_html,'w') as f:
            f.write(new_page)


col75_redund0.99
cp ../data/processed/align_fromErick/evc/col75_redund0.99/couplings/col75_redund0.99_evzoom.json ../data/processed/align_fromErick/evzoom/data/col75_redund0.99_evzoom.json
../data/processed/align_fromErick/evzoom/example/col75_redund0.99.html


FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/align_fromErick/evzoom/example/evzoom.html'

## After looking at the contact matrices, very little is different , so using column gap threshold= 75 and theta=0.9 parameters as the representative one
(ask for other parameters if desired)

In [ ]:
col_thresh = 75
red_thresh = 0.99
name = f"col{col_thresh}_redund{red_thresh}"
new_name = "sars_cov2_S_protein"

name = f"col{col_thresh}_redund{red_thresh}"
print(name)
#Copy over json
coupling_prefix = os.path.join(ev_dir, name,"couplings",name)
json_f = coupling_prefix + '_evzoom.json'
out_json_f = evzoom_dir+"/data/" + os.path.basename(json_f).replace(name,new_name )

cmd = f'cp {json_f} {out_json_f}'
print(cmd)
os.system(cmd)

# Create the html file
tmp_html = join(evzoom_dir, 'example','evzoom.html')
out_html = join(evzoom_dir, 'example',new_name+'.html')
print(out_html)
with open(tmp_html, "r") as f:
    page = f.read()
new_page=""
line = f'<div id="evzoom-viewer" data-couplings=../data/{basename(out_json_f)}> </div> '
for i in page.split("\n"):
    if 'data-couplings' in i:
        new_page = new_page + line + "\n"
    else:
        new_page = new_page + i + "\n"
new_page = new_page.strip("\n")
with open(out_html,'w') as f:
    f.write(new_page)

            


## Create contact matrix for cn and probability

In [ ]:
## 
column_thresholds = [75]
redundant_thresholds = [0.99]

files_to_run = []

for col_thresh in column_thresholds:
    for red_thresh in redundant_thresholds:
        gene_of_interest = f"col{col_thresh}_redund{red_thresh}"
        print(gene_of_interest)
        align_prefix = os.path.join(ev_dir,gene_of_interest,"align",gene_of_interest)
        coupling_prefix = os.path.join(ev_dir, gene_of_interest,"couplings",gene_of_interest)

        construct_monomer_contact_matrix(gene_of_interest, 
                                         input_dir = ev_dir, 
                                         to_plot=True, 
                                         f_save_fig = join(contact_dir, gene_of_interest+"_pval"),
                                         f_save = join(contact_dir, gene_of_interest+"_pval"), 
                                         col='probability', conservation=True) 
        
        construct_monomer_contact_matrix(gene_of_interest, 
                                         input_dir = ev_dir, 
                                         to_plot=True, 
                                         f_save_fig = join(contact_dir, gene_of_interest),
                                         f_save = join(contact_dir, gene_of_interest), 
                                         col='cn', conservation=True) 
